## Using ipywidgets to visualize:
 - eigenmodes of the complex laplacian
 - effects of alpha parameter on eigenmodes
 - effects of frequency on eigenmodes

In [1]:
# for the widget
from ipywidgets import interactive, widgets, fixed
from surfer import Brain as brainsurface
import os
import nibabel as nib

# number things
import numpy as np
import pandas as pd
import h5py
import matplotlib.pyplot as plt

# spectrome imports
from spectrome.brain import Brain
from spectrome.utils import functions, path
from spectrome.forward import eigenmode, get_complex_laplacian, runforward

In [2]:
# Some house keeping
hcp_dir = "/home/axiezai/lab/brain-scfc/spectrome/spectrome/data"

# Define frequency range of interest
fmin = 2 # 2Hz - 45Hz signal range, filter for this with hbp
fmax = 45
fvec = np.linspace(fmin,fmax,44)

# create HCP brain object:
HCP_brain = Brain.Brain()
HCP_brain.add_connectome(hcp_dir)
HCP_brain.reorder_connectome(HCP_brain.connectome, HCP_brain.distance_matrix)
HCP_brain.bi_symmetric_c()
HCP_brain.reduce_extreme_dir()

In [3]:
%gui qt

# set up Pysurfer variables
subject_id = "fsaverage"
hemi = "lh"
surf = "inflated"
"""
Bring up the visualization.
"""

surfacebrain = brainsurface(subject_id, hemi, surf, background = "white", title = "Eigenmodes of complex Laplacian")
surfacebrain.add_text(x = 0.5, y=0.9, text = "Eig Number , frequency = , alpha = ", name = "eiglabel")

"""
Read in the automatic parcellation of sulci and gyri.
"""

hemi_side = "lh"
aparc_file = os.path.join(os.environ["SUBJECTS_DIR"],
                          subject_id, "label",
                          hemi_side + ".aparc.annot")
labels, ctab, names = nib.freesurfer.read_annot(aparc_file)

In [4]:
def em_widget(brain, fvec, labels, f_in, alpha, em_num):
    freq_index = np.abs(fvec-f_in).argmin()
    omega = 2*np.pi*fvec[freq_index]
    brain.add_laplacian_eigenmodes(w = omega, alpha = alpha, speed = 10, num_ev = 86)
    
    # get cortical eigenmodes
    cortical_eigs = brain.norm_eigenmodes[0:68,:]
    lh_cortical_eigs = cortical_eigs[0:34, em_num-1]
    # we need to pad our eigenmode so that len(em) = len(names) for ipywidget and pysurf
    lh_cortical_padded = np.insert(lh_cortical_eigs, [0,3],[0,0])
    
    lh_cort_surface = lh_cortical_padded[labels] # transform the padded eigenmode into appropriate surface plot labels
    surfacebrain.add_data(lh_cort_surface, hemi="lh", thresh=0, colormap="jet", remove_existing = True)
    surfacebrain.update_text(text = "Eig Number %s, frequency = %s, alpha = %s" %(em_num, f_in, alpha), name = "eiglabel")
    return cortical_eigs

In [6]:
interactive(em_widget, brain = fixed(HCP_brain), fvec = fixed(fvec), labels = fixed(labels),
           f_in = widgets.IntSlider(min=2, max = 40, step = 1, value = 10, description = "Frequency", continuous_update = False),
           alpha = widgets.FloatSlider(min = 0.1, max = 1, step = 0.1, value = 0.5, description = "Alpha", continuous_update = False),
            em_num = widgets.BoundedIntText(value = 1, min = 1, max = 86, step = 1, description = "Eigenmode", continuous_update = False, disabled = False)
           )

interactive(children=(IntSlider(value=10, continuous_update=False, description='Frequency', max=40, min=2), Fl…

In [14]:
def em_alpha_plot(brain,fvec,labels,alpha):
    freq_index = np.abs(fvec-10).argmin()
    omega = 2*np.pi*fvec[freq_index]
    brain.add_laplacian_eigenmodes(w = omega, alpha = alpha, speed = 10, num_ev = 86)
    #threshold
    brain.binary_eigenmodes = np.where(
    brain.norm_eigenmodes > 0.6, 1, 0
    )
    
    # plot the eigenmodes
    plt.imshow(brain.binary_eigenmodes,cmap=plt.cm.Blues)
    plt.xlabel('Eigenmodes')
    plt.colorbar()

In [15]:
interactive(em_alpha_plot, brain = fixed(HCP_brain), fvec = fixed(fvec), labels = fixed(labels),
           alpha = widgets.FloatSlider(min=0.5, max = 15, step = 0.5, value = 1, description = "Alpha", continuous_update = False))

interactive(children=(FloatSlider(value=1.0, continuous_update=False, description='Alpha', max=15.0, min=0.5, …